[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/facebookresearch/droidlet/blob/master/tutorials/how_to_build_a_simple_agent.ipynb)

# How to build your own agent

<p align="center">
   <img src="https://craftassist.s3-us-west-2.amazonaws.com/pubr/demo.gif" />
</p>

## Build a simple agent
In this tutorial, we will build a simple agent that catches a randomly moving bot in a 5x5 grid world.  The goal is to understand the high level organization of the droidlet agent.


## Control logic

The basic droidlet agent is made up of four major components: a perceptual API, a memory system, a controller, and a task queue. In each iteration of the event loop, the agent will run perceptual modules, updating the memory system with what it perceives, maybe place tasks into the task queue, and lastly, pop all finished tasks and then step the highest priority task.

A typical event loop is as follows: 

> **while** True **do**
>> run [perceptual modules](https://facebookresearch.github.io/droidlet/perception.html), update [memory](https://facebookresearch.github.io/droidlet/memory.html)
>>
>> step [controller](https://facebookresearch.github.io/droidlet/controller.html)
>>
>> step highest priority [task](https://facebookresearch.github.io/droidlet/tasks.html)

<!---

### **Perception**
Perception modules

 is where the agent perceives the world it resides. Most of the perceptual modules in our example agents are visual: e.g. object detection and instance segmentation. You can customize your own perception modules and have it registered in the agent.

All the information perception modules receive should go into agent's memory system. 


### **Memory System**
Memory system serves as the interface for passing information between the various components of the agent. It consists of an AgentMemory object which is the entry point to the underlying SQL database and some MemoryNodes which represents a particular entity or event. It stores and organizes information like: 
- player info
- time info
- program info
- task info
- etc.

### **Controller**

Controller is where agent interpret commands, carry out dialogues and place tasks on the task stack.

### **Task queue**
Task queue stores tasks, which are (mostly) self-contained lower-level world interactions (e.g. Move, Point). For each event loop, one task is poped out of task queue and got executed by the agent.
-->

### Extend BaseAgent
---

The first you need to do is to extend the BaseAgent class and overwrite the following functions:



In [2]:
# grid_agent.py
class GridAgent(BaseAgent):
    def __init__(self, world=None, opts=None):
        self.world = world
        self.pos = (0, 0, 0)
        super(GridAgent, self).__init__(opts)

    def init_memory(self):
        pass

    def init_perception(self):
        pass

    def init_controller(self):
        pass
    
    def perceive(self):
        pass

    def get_incoming_chats(self):
        pass

    def controller_step(self):
        pass
    
    def task_step(self, sleep_time=5):
        pass
    
    def handle_exception(self, e):
        pass
    
    def send_chat(self, chat):
        pass

NameError: name 'BaseAgent' is not defined

We will go over each components in the following sections.

### Create a simple 5x5 grid world
---

Note that in the above ```___init___``` function we are passing a world to GridAgent, which is a simulated 5x5(x1) gridworld which hosts our agent. We also put a simple bot named "target" in it; our agent will need to catch it.

In [3]:
# world.py
Bot = namedtuple("Bot", "entityId, name, pos, look")

class World:
    def __init__(self, opts=None, spec=None):
        target = Bot(1977, "target", Pos(3, 4, 0), Look(0, 0))
        self.bots = [target]
    
    def get_bots(self, eid=None):
        bots = self.bots if eid is None else [b for b in self.bots if b.entityId == eid]
        return bots
    
    def remove_bot(self, eid):
        self.bots[:] = [b for b in self.bots if b.entityId != eid]

NameError: name 'namedtuple' is not defined

### Heuristic Perception
---

In order to catch the target, our agent needs to keep track of its location. We add a heuristic perception module that gets the position of all bots in the world and put them into memory.    

In a more sophisticated agent, the perceptual models might be mediated by more in-depth heuristics or machine-learned models; but they would interface the Memory system in a similar way.

In [ ]:
# heuristic_perception.py
class HeuristicPerception:
    def __init__(self, agent):
        self.agent = agent

    def perceive(self):
        bots = self.agent.world.get_bots()
        for bot in bots:
            bot_node = self.agent.memory.get_player_by_eid(bot.entityId)
            if bot_node is None:
                memid = PlayerNode.create(self.agent.memory, bot)
                bot_node = PlayerNode(self.agent.memory, memid)
                self.agent.memory.tag(memid, "bot")
            bot_node.update(self.agent.memory, bot, bot_node.memid)
            
        

### Memory Module
---

To store and organize all the information, the agent needs a Memory Module. Here we just use [AgentMemory](https://facebookresearch.github.io/droidlet/memory.html#base_agent.sql_memory.AgentMemory) of base_agent and use [PlayerNode](https://facebookresearch.github.io/droidlet/memory.html#memorynodes) to represent the bot entity. You can also extend them and define your own Memory Nodes.

### Tasks
---

A [Task](https://facebookresearch.github.io/droidlet/tasks.html) is a world interaction whose implementation might vary from platform to platform. 

<!--It usually consists of a target (e.g. a certain position the agent wants to move to, or an entity the agent wants to destroy) and a stop condition. We usually break it down to several small steps and do one at a time in step function until it is finished (stop condition is met). -->

#### **Simple Catch Task**

We are going to create a simple Catch Task for our agent. We break it into two smaller subtasks: a Move Task and a Grab Task. 

In Move Task, the agent will simply head to a given position. The stop condition is when the agent is at the exact location of the target. It will move one block at a time to get close to the target until the stop condition is met.

In Grab Task, the agent will simply grab the target physically. The stop condition is when the target has disappeared from the world.

In [ ]:
# tasks.py
class Move(Task):
    def __init__(self, agent, task_data):
        super(Move, self).__init__()
        self.target = task_data["target"]
    
    def step(self, agent):
        super().step(agent)
        if self.finished:
            return
        agent.move(self.target[0], self.target[1], self.target[2])
        self.finished = True


class Grab(Task):
    def __init__(self, agent, task_data):
        super(Grab, self).__init__()
        self.target_eid = task_data["target_eid"]

    def step(self, agent):
        super().step(agent)
        if self.finished:
            return

        if len(agent.world.get_bots(eid=self.target_eid)) > 0:
            agent.catch(self.target_eid)
        else:
            self.finished = True



class Catch(Task):
    def __init__(self, agent, task_data):
        super(Catch, self).__init__()
        self.target_memid = task_data["target_memid"]
    
    def step(self, agent):
        super().step(agent)
        if self.finished:
            return

        # retrieve target info from memory:
        target_mem = agent.memory.get_mem_by_id(self.target_memid)
                    
        # first get close to the target, one block at a time
        tx, ty, tz = target_mem.get_pos()
        x, y, z = agent.get_pos()
        if np.linalg.norm(np.subtract((x, y, z), (tx, ty, tz))) > 0.:
            if x != tx:
                x += 1 if x - tx < 0 else -1
            else:
                y += 1 if y - ty < 0 else -1
            move_task = Move(agent, {"target": (x, y, z)})
            agent.memory.add_tick()
            self.add_child_task(move_task, agent)
            return

        # once target is within reach, catch it!
        grab_task = Grab(agent, {"target_eid": target_mem.eid})
        agent.memory.add_tick()
        self.add_child_task(grab_task, agent)
        self.finished = True

### Controller
---

The [Controller](https://facebookresearch.github.io/droidlet/controller.html) decides which Tasks (if any) to put on the stack.  In the [craftassist](https://github.com/facebookresearch/droidlet/blob/main/craftassist/agent/craftassist_agent.py) and [locobot](https://github.com/facebookresearch/droidlet/blob/main/locobot/agent/locobot_agent.py) agents, the controller is itself a modular, multipart system.  

In this tutorial, to keep things simple and self contained, the controller will just push the Catch task onto the stack.   

For more in-depth discussion about Controllers we use, look [here](https://facebookresearch.github.io/droidlet/controller.html)


In [ ]:
# grid_agent.py
class GridAgent(BaseAgent):
    ...
    ...

    def controller_step(self):
        bot_memids = self.memory.get_memids_by_tag("bot")
        if self.memory.task_stack_peek() is None:
            if bot_memids:            
                task_data = {"target_memid": bot_memids[0]}
                self.memory.task_stack_push(Catch(self, task_data))
            else:
                exit()


### Task Step
---

Here the agent steps the topmost Task on the Stack.


In [ ]:
# grid_agent.py
class GridAgent(BaseAgent):
    ...
    ...
    
    def task_step(self, sleep_time=5):
        # clear finsihed tasks from stack
        while (
            self.memory.task_stack_peek() and self.memory.task_stack_peek().task.check_finished()
        ):
            self.memory.task_stack_pop()

        # do nothing if there's no task
        if self.memory.task_stack_peek() is None:
            return

        # If something to do, step the topmost task
        task_mem = self.memory.task_stack_peek()
        if task_mem.memid != self.last_task_memid:
            self.last_task_memid = task_mem.memid
        task_mem.task.step(self)
        self.memory.task_stack_update_task(task_mem.memid, task_mem.task)

### Put it together
---

In [ ]:
# grid_agent.py

class GridAgent(BaseAgent):
    def __init__(self, world=None, opts=None):
        self.world = world
        self.last_task_memid = None
        self.pos = (0, 0, 0)
        super(GridAgent, self).__init__(opts)

    def init_memory(self):
        self.memory = AgentMemory()

    def init_perception(self):
        self.perception_modules = {}
        self.perception_modules['heuristic'] = HeuristicPerception(self)

    def init_controller(self):
        pass
    
    def perceive(self):
        self.world.step() # update world state
        for perception_module in self.perception_modules.values():
            perception_module.perceive()

    def controller_step(self):
        bot_memids = self.memory.get_memids_by_tag("bot")
        if self.memory.task_stack_peek() is None:
            if bot_memids:            
                task_data = {"target_memid": bot_memids[0]}
                self.memory.task_stack_push(Catch(self, task_data))
                logging.info(f"pushed Catch Task of bot with memid: {bot_memids[0]}")
            else:
                exit()
    
    def task_step(self, sleep_time=5):
        while (
            self.memory.task_stack_peek() and self.memory.task_stack_peek().task.check_finished()
        ):
            self.memory.task_stack_pop()

        # do nothing if there's no task
        if self.memory.task_stack_peek() is None:
            return

        # If something to do, step the topmost task
        task_mem = self.memory.task_stack_peek()
        if task_mem.memid != self.last_task_memid:
            logging.info("Starting task {}".format(task_mem.task))
            self.last_task_memid = task_mem.memid
        task_mem.task.step(self)
        self.memory.task_stack_update_task(task_mem.memid, task_mem.task)
        self.world.visualize(self)

    """physical interfaces"""
    def get_pos(self):
        return self.pos
    
    def move(self, x, y, z):
        self.pos = (x, y, z)
        return self.pos
    
    def catch(self, target_eid):
        bots = self.world.get_bots(eid=target_eid)
        if len(bots) > 0:
            bot = bots[0]
            if np.linalg.norm(np.subtract(self.pos, bot.pos)) <1.0001:
                self.world.remove_bot(target_eid)

### Run the agent

To run the agent, you need to create a runtime populated with files we just created. Luckily we have already prepared one for you. Simply run the following command to pull it and install required packages.

In [ ]:
!git clone https://github.com/facebookresearch/droidlet.git && cd examples/grid && pip install -r requirements.py

### Run it now!

In [ ]:
%run agent/grid_agent.py